In [121]:
%%capture

# Turn off warnings
import warnings

warnings.filterwarnings("ignore")

import functools

# Normal packages
import geopandas as gpd
import ipywidgets as widgets
import numpy as np
import pandas as pd

# Format
from babel.numbers import format_currency

# Display
from IPython.display import HTML, Image, Markdown, display, display_html
from ipywidgets import AppLayout, Layout
from ipywidgets import interact

# Settings
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{:,.2f}".format

# GCS, del later since this will presumbly be read from a script that cleans up the data
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"
FILE = "fake_data.xlsx"

# My utilities
import _utils

In [122]:
# DEL LATER
import altair as alt
import panel as pn
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp

In [123]:
# Read in file
df_statewide = pd.read_excel(f"{GCS_FILE_PATH}{FILE}", sheet_name="fake")

#### Manipulation

In [124]:
csis = df_statewide[
    [  "project_name",
     "project_description",
        "mode_shift__csis_",
        "vmt__csis_",
        "dac_local_community_needs__csis_",
        "public_engagement__csis_",
        "safety__csis_",
        "zev__csis_",
        "climate_resiliency__csis_",
        "natural_resources_and_ecosystems__csis_",
        "infill_development_and_land_use__csis_",
  
    ]
]

In [125]:
csis = pd.melt(
    csis,
    id_vars=[
        "project_name",
        "project_description"
    ],
    value_vars=[
        "mode_shift__csis_",
        "vmt__csis_",
        "dac_local_community_needs__csis_",
        "public_engagement__csis_",
        "safety__csis_",
        "zev__csis_",
        "climate_resiliency__csis_",
        "natural_resources_and_ecosystems__csis_",
        "infill_development_and_land_use__csis_",
   
    ],
)

In [126]:
# Remove underscores off of old column names
csis["variable"] = csis["variable"].str.replace("_", " ").str.strip().str.title()
csis = csis.rename(columns={"variable": "csis", "value": "csis values"})

In [127]:
# Second subset
csis_2 = df_statewide[
    [
        "total_project_cost__$1,000_",
        "pa_ed_cost__$1,000_",
        "ps_e_cost__$1,000_",
        "row_cost__$1,000_",
        "con_cost__$1,000_",
        "non_infrastructure___plan_cost__$1,000_",
        "total_unfunded_need__$1,000_",
        "project_name",
        "project_description",
    ]
]

In [128]:
# Melt
csis_2 = pd.melt(
    csis_2,
    id_vars=[
        "project_name",
        "project_description",
    ],
    value_vars=[
        "total_project_cost__$1,000_",
        "pa_ed_cost__$1,000_",
        "ps_e_cost__$1,000_",
        "row_cost__$1,000_",
        "con_cost__$1,000_",
        "non_infrastructure___plan_cost__$1,000_",
        "total_unfunded_need__$1,000_",
    ],
)

In [129]:
# Change names
csis_2 = csis_2.rename(columns={"variable": "monetary", "value": "monetary values"})
csis_2["monetary"] = csis_2["monetary"].str.replace("_", "  ").str.title()

In [130]:
# Final Merge
final = pd.merge(csis, csis_2, how="inner", on=["project_name", "project_description"])

### Ipywidgets 

In [131]:
my_list_of_projects = df_statewide.project_name.sort_values().unique().tolist()

In [132]:
subset = df_statewide[['project_name','full_county_name','district_full_name',  'project_description','total_project_cost__$1,000_', 'total_unfunded_need__$1,000_', 
                       'csis_total_score__out_of_45_',]]

In [133]:
def styled_df(df):
    display(
    HTML(
        "<div style='height: 500px; overflow: auto; width: 800px'>"
        + (
            (df)
            .style.set_properties(**{"background-color": "white", "font-size": "10pt",})
            .set_table_styles([dict(selector="th", props=[("text-align", "center")])])
            .set_properties(**{"text-align": "center"})
            .hide(axis="index")
            .render()
        )
        + "</div>"
    ))


In [134]:
def show_df(value):
    df_statewide
    return  styled_df(_utils.clean_up_columns((subset[subset.project_name==value]))) 

In [139]:
test_widget = widgets.Select(
    options=my_list_of_projects,
    value='Napa Valley Forward Sr 29 Up Valley Roundabout Project',
    description='Project:',
    disabled=False
)

In [140]:
interact(show_df, value = test_widget)

interactive(children=(Select(description='Project:', index=334, options=('#Raisefontana', '180 West Extension'…

<function __main__.show_df(value)>